# Exploratory Data Analysis

## Libraries and data import

In [ ]:
!pip install -q -r ../requirements.txt

In [ ]:
import os, sys

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.io as pio
pio.templates.default = 'seaborn'

sys.path.append(os.path.abspath('..'))

In [ ]:
DATA_DIR = '/datasets/data2021-09-20/'

lesions_file = 'melanoma_lesion-info_organ-overlap_2021-09-17_anonymized_cleaned_all.csv'
lesion_mapping_file = 'melanoma_lesion_mapping_2021-09-20_anonymized.csv'
patients_file = 'melanoma_patient-level_summary_anonymized.csv'
studies_file = 'melanoma_study_level_summary_anonymized.csv'

## Lesions

### Dataset information

The lesions dataset contains the following features:

* Each patient is attributed to a unique `gpcr_id [int]` 
* Each study is identified relative to the start of treatment with `study_name [str]` and the processed `study_phase [int]`
* `roi_id [int]` is an identifier for the lesion's ROI (Region Of Interest)
* `roi_name [str]` is a textual identifier for the lesion
* `lesion_label_id [int]` is an identifier for the lesion's label
* `pars_bodypart_petct [str]`, `pars_region_petct [str]`, `pars_subregion_petct [str]`, `pars_laterality_petct [str]` are categorical values output by PARS that help identify the location of the lesion
* `pars_classification_petct [str]` is a categorical variable (either `benign` or `suspicious`)
* `vol_ccm [float]` is the lesion volume in cubic centimeters
* `max_suv_val [float]`, `mean_suv_val [float]`, `min_suv_val [float]`, and `sd_suv_val [float]` are relative to the lesions [SUV  (Standardized Uptake Values)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3026294/)
* `is_malignant [bool]` is the boolean value of `pars_classification_petct == 'suspicious'`
*  `assigned_organ [str]` is the lesion's assigned organ (which is PARS output)

### Preprocessing

In [ ]:
from src.utils import extract_study_phase

ModuleNotFoundError: No module named 'torch_geometric'

In [ ]:
lesions = pd.read_csv(os.path.join(DATA_DIR, lesions_file))
lesions['study_phase'] = lesions.study_name.apply(extract_study_phase)
lesions = lesions[lesions.pars_classification_petct != 'benign']

# Sorting by PARS classification allows for visualization consistency
lesions.sort_values(by='pars_classification_petct', inplace=True)
n_lesions, f_lesions = lesions.shape

print(f'Imported {n_lesions} lesions with {f_lesions} features. Null values: {lesions.isna().sum().sum()}')

Imported 3251 lesions with 18 features. Null values: 0


In [ ]:
lesions.head()

,gpcr_id,study_name,roi_id,roi_name,lesion_label_id,pars_bodypart_petct,pars_region_petct,pars_subregion_petct,pars_laterality_petct,pars_classification_petct,vol_ccm,max_suv_val,mean_suv_val,min_suv_val,sd_suv_val,is_malignant,assigned_organ,study_phase
0,34610002,pre-01,0,Muscles1,1,lower limb,muscles,upper leg group,right,suspicious,1.00,5.58,3.37,2.36,0.92,True,other_lowerlimb,-1
16135,34610104,post-01,23,Liver1,24,abdomen,liver,not specified,right,suspicious,1.03,3.66,3.13,2.82,0.21,True,liver,1
16137,34610104,post-01,25,Liver2,26,abdomen,liver,not specified,right,suspicious,2.91,7.23,4.16,3.05,0.98,True,liver,1
16138,34610104,post-01,26,Liver3,27,abdomen,liver,not specified,left,suspicious,3.89,4.41,2.96,2.51,0.43,True,liver,1
16142,34610104,post-01,30,Spleen1,31,abdomen,spleen,not specified,not specified,suspicious,13.54,16.34,10.14,6.87,2.08,True,spleen,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8143,34610044,post-03,49,Bones12,50,thorax,bones,ribs,not specified,suspicious,0.51,3.43,2.35,1.81,0.46,True,other_thorax,3
8144,34610044,post-03,50,Pleura2,51,thorax,pleura,not specified,not specified,suspicious,5.82,11.60,7.23,4.89,1.65,True,other_thorax,3
8145,34610044,post-03,51,LymphNodes11,52,thorax,lymph nodes,IASLC station 10,left,suspicious,0.66,4.59,3.86,3.54,0.30,True,lymphnode_thorax,3
8135,34610044,post-03,39,Bones9,40,thorax,bones,ribs,not specified,suspicious,0.88,4.20,2.60,1.78,0.66,True,bones_thorax,3


In [ ]:
lesions_labels = {
    'vol_ccm': 'Lesion volume (ccm)',
    'pars_classification_petct': 'PARS lesion classification',
    'study_phase': 'Study phase (relative to treatment start)',
    'assigned_organ': 'Assigned organ'
}

In [ ]:
top_h_legend = dict(orientation='h', yanchor="bottom", y=1.02)

### Visualisations

In [ ]:
px.histogram(lesions, x='study_phase', color='pars_classification_petct',
             labels=lesions_labels) \
    .update_layout(legend=top_h_legend, yaxis_title='Lesions')

In [ ]:
descending_organs = list(lesions.groupby('assigned_organ').size().sort_values(ascending=False).index)

px.histogram(lesions, x='assigned_organ', color='pars_classification_petct', 
             category_orders={'assigned_organ': descending_organs},
             labels=lesions_labels) \
    .update_layout(legend=top_h_legend, yaxis_title='Lesions')

In [ ]:
avg_lesion_vol = lesions.groupby(['gpcr_id', 'study_phase', 'pars_classification_petct']) \
    .vol_ccm.mean().to_frame('vol_ccm').reset_index() \

px.box(avg_lesion_vol, x='study_phase', y='vol_ccm', color='pars_classification_petct', 
       labels={**lesions_labels, 'vol_ccm': 'Average lesion volume (ccm)'}) \
    .update_layout(legend=top_h_legend)

In [ ]:
location_hierarchy = ['pars_bodypart_petct', 'pars_region_petct', 'pars_subregion_petct', 'pars_classification_petct']
locations = lesions.groupby(location_hierarchy).size().to_frame('lesions').reset_index()

px.sunburst(locations, path=location_hierarchy, values='lesions') \
    .update_layout(title='Sunburst chart of lesion location')

## Studies

### Dataset information

* Each patient is attributed to a unique `gpcr_id [int]` 
* Each study is identified relative to the start of treatment with `study_name [str]` and the processed `study_phase [int]`
* `is_before_treatment [bool]`, `is_during_treatment [bool]`, and `is_after_treatment_end [bool]` are boolean values that explain when the study occured relative to treatment start and end
* `nth_before_treatment [float]`, `nth_after_treatment_start [float]`, `nth_during_treatment [float]`, and `nth_after_treatment_end [float]` is the scan number relative to treatment start and end
* `n_days_to_treatment_start [int]` and `n_days_to_treatment_end [int]` are the number of days to treatment start and end
* `is_malignant [int]` is the aggregate number of malignant lesions in the exam, which is renamed to `malignant_lesions [int]` during preprocessing
* Boolean values about segmentation existance are contained in `brain_seg_exists [bool]`, `bones_seg_exists [bool]`, `spleen_seg_exists [bool]`, `aorta_seg_exists [bool]`, `heart_seg_exists [bool]`, `kidney_right_seg_exists [bool]`, `kidney_left_seg_exists [bool]`, `lung_right_seg_exists [bool]`, `lung_left_seg_exists [bool]`, and `liver_seg_exists [bool]`

### Preprocessing

In [ ]:
studies = pd.read_csv(os.path.join(DATA_DIR, studies_file))
studies.rename(columns={'is_malignant': 'malignant_lesions'}, inplace=True)
studies['study_phase'] = studies.study_name.apply(extract_study_phase)

n_studies, f_studies = studies.shape

print(f'Imported {n_studies} studies with {f_studies} features. Null values: {studies.isna().sum().sum()}')

Imported 472 studies with 23 features. Null values: 1043


In [ ]:
studies.head()

,gpcr_id,study_name,is_before_treatment,is_during_treatment,is_after_treatment_end,nth_before_treatment,nth_after_treatment_start,nth_during_treatment,nth_after_treatment_end,n_days_to_treatment_start,...,bones_seg_exists,spleen_seg_exists,aorta_seg_exists,heart_seg_exists,kidney_right_seg_exists,kidney_left_seg_exists,lung_right_seg_exists,lung_left_seg_exists,liver_seg_exists,study_phase
0,34610001,pre-02,True,False,False,2.0,NaN,NaN,NaN,-43,...,True,True,True,True,True,True,True,True,True,-2
1,34610001,pre-01,True,False,False,1.0,NaN,NaN,NaN,-7,...,True,True,True,True,True,True,True,True,True,-1
2,34610001,post-01,False,False,True,NaN,1.0,NaN,1.0,87,...,True,True,True,True,True,True,True,True,True,1
3,34610001,post-02,False,False,True,NaN,2.0,NaN,2.0,183,...,True,True,True,True,False,False,True,True,True,2
4,34610001,post-03,False,False,True,NaN,3.0,NaN,3.0,275,...,True,True,True,True,True,True,True,True,True,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,34610149,post-02,False,False,True,NaN,2.0,NaN,2.0,136,...,True,True,True,True,True,True,True,True,True,2
468,34610150,post-01,False,False,True,NaN,1.0,NaN,1.0,63,...,False,True,True,True,True,True,True,True,True,1
469,34610150,post-02,False,False,True,NaN,2.0,NaN,2.0,147,...,True,True,True,True,True,True,True,True,True,2
470,34610150,post-03,False,False,True,NaN,3.0,NaN,3.0,247,...,True,True,True,True,True,True,True,True,True,3


In [ ]:
studies_labels = {
    **lesions_labels,
    'response': 'Immunotherapeutic response',
    'tmtv': 'Total Metabolic Tumor Volume (ccm)',
    'baseline': 'Baseline TMTV (ccm)'
}

#### Creating temporary labels using TMTV (Total Metabolic Tumor Volume)


* If TMTV is non-zero, but lower compared to baseline: Partial response
* If TMTV is zero / no lesions detected: Complete response
* If TMTV > 'best' previous response: progressive disease.
     'Best' previous responds means minimum TMTV in previous time points
* Otherwise: stable disease

e.g. for 4 timepoints, TMTV in arbitrary units, just to illustrate the trend:
- TMTV = 10, 11, 14, 11 -> baseline, progressing, progressing, progressing, progressing
- TMTV = 10, 8, 5, 4, 4   -> baseline, responding, responding, responding, responding/stable (not sure)
- TMTV = 10, 8, 5, 8, 10 -> baseline, responding, responding, progressing, progressing

In [ ]:
def classify_response(row):
    # Compare current vs. baseline
    if row.tmtv < row.baseline:
        return 'response'
    elif row.tmtv == row.baseline:
        return 'baseline'
    else:
        return 'progression'

In [ ]:
# Compute TMTV by summing `vol_ccm` per study
labels = lesions.groupby(['gpcr_id', 'study_phase']).vol_ccm.sum().to_frame('tmtv').reset_index()

# Find `labels` id of each patient's baseline TMTV
baseline_idx = labels.groupby('gpcr_id').study_phase.idxmin().to_dict()
labels['baseline'] = labels.gpcr_id.apply(
    lambda i: labels.loc[baseline_idx[i]].tmtv)
labels['response'] = labels.apply(classify_response, axis=1)

print(f'Response computed for {labels.shape[0]} studies: \n{labels.response.value_counts()}')

Response computed for 366 studies: 
progression    124
response       123
baseline       119
Name: response, dtype: int64


In [ ]:
labels.head()

,gpcr_id,study_phase,tmtv,baseline,response
0,34610002,-1,36.38,36.38,baseline
1,34610002,1,255.99,36.38,progression
2,34610004,-2,16.15,16.15,baseline
3,34610004,-1,16.73,16.15,progression
4,34610004,1,23.05,16.15,progression
...,...,...,...,...,...
361,34610148,2,555.24,35.91,progression
362,34610148,3,1.15,35.91,response
363,34610149,-1,3.03,3.03,baseline
364,34610149,1,26.36,3.03,progression


### Visualisations

In [ ]:
px.scatter(labels, x='study_phase', y='tmtv', color='response',
           labels=studies_labels) \
    .update_layout(legend=top_h_legend)

In [ ]:
# Checking post-1 label imbalance
post_1_response = labels[labels.study_phase == 1].response.value_counts()

print(f'Response imbalance for post-1 studies: \n{post_1_response}')

Response imbalance for post-1 studies: 
baseline       43
progression    34
response       18
Name: response, dtype: int64


In [ ]:
px.histogram(labels[labels.study_phase == 1], x='tmtv', color='response',
             nbins= 40, marginal="rug",
             labels=studies_labels) \
    .update_layout(legend={**top_h_legend, 'y': 1.0},
                   yaxis_title='Studies (count)',
                   title='Post-1 study phase response distribution') \
    .update_traces(opacity=0.7)

## Patients

### Dataset information

### Preprocessing

In [ ]:
patients = pd.read_csv(os.path.join(DATA_DIR, patients_file))

patients['age_at_treatment_start_in_years'] = \
    patients.age_at_treatment_start_in_years.apply(
        lambda a: 90 if a == '90 or older' else int(a))

n_patients, f_patients = patients.shape

print(f'Imported {n_patients} patients with {f_studies} features. Null values: {patients.isna().sum().sum()}')

Imported 129 patients with 9 features. Null values: 0


In [ ]:
patients.head()

,gpcr_id,age_at_treatment_start_in_years,duration_treatment_in_days,death_event_observed,survival_in_days,n_imgs_before_treatment,n_imgs_during_treatment,n_imgs_after_treatment_end,n_imgs_after_treatment_start
0,34610039,64,63,False,768,1,0,1,1
1,34610116,80,22,False,1312,2,0,4,4
2,34610117,55,0,False,1145,0,0,4,4
3,34610118,68,63,True,639,2,0,6,6
4,34610042,52,60,False,707,0,0,7,7
...,...,...,...,...,...,...,...,...,...
124,34610037,88,261,True,747,1,2,3,5
125,34610114,77,21,False,1343,1,0,4,4
126,34610005,51,63,False,1619,0,0,10,10
127,34610115,71,0,False,1690,0,0,3,3


In [ ]:
patients_labels = {
    **studies_labels,
    'age_at_treatment_start_in_years': 'Age (treatment start, years)',
    'duration_treatment_in_days': 'Treatment duration (days)',
    'death_event_observed': 'Death observed',
    'survival_in_days': 'Survival time (days)',
    'time': 'Time (days)',
    'survival_probability': 'Survival probability'
}

### Visualisations

In [ ]:
px.histogram(patients, x='age_at_treatment_start_in_years',
             labels=patients_labels) \
    .update_layout(legend=top_h_legend, yaxis_title='Patients (count)')

In [ ]:
px.histogram(patients, x='duration_treatment_in_days', facet_col='death_event_observed',
             labels=patients_labels) \
    .update_layout(legend=top_h_legend, yaxis_title='Patients (count)')

In [ ]:
px.scatter(patients, x='age_at_treatment_start_in_years', 
           y='survival_in_days', color='death_event_observed',
           labels=patients_labels) \
    .update_layout(legend=top_h_legend)

In [ ]:
!pip install scikit-survival
from sksurv.nonparametric import kaplan_meier_estimator

# Kaplan-Meier
time, s_prob = kaplan_meier_estimator(patients.death_event_observed, patients.survival_in_days)
km_df = pd.DataFrame(np.array([time, s_prob]).T, columns=['time', 'survival_probability'])

     |████████████████████████████████| 2.4 MB 18.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 147 kB 40.0 MB/s 
     |████████████████████████████████| 471 kB 49.5 MB/s 
     |████████████████████████████████| 212 kB 37.7 MB/s 
     |████████████████████████████████| 941 kB 46.3 MB/s 
  Created wheel for scikit-survival: filename=scikit_survival-0.15.0.post0-cp37-cp37m-linux_x86_64.whl size=2560979 sha256=bbcfac7452de058ba554cd85be089fc6eb0d4050dfbcfc605b2e2fe87a56c570
  Stored in directory: /root/.cache/pip/wheels/18/8d/37/1c15bd2cfef9e7c5c81f3742b66f6656d99d29c104a10a8b68
Successfully built scikit-survival


In [ ]:
px.line(km_df, x='time', y='survival_probability', line_shape='hv',
        labels=patients_labels)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7360151b-146e-499a-b01b-835ca18c34fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>